In [3]:
import os
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
from dotenv import load_dotenv
from IPython.display import Markdown
from dotenv import load_dotenv
load_dotenv()

True

### Dataset is 1,200 DS related job posting from Indeed

In [4]:
df = pd.read_parquet("cluster_test_results.parquet")

In [5]:
print(df.shape)
df.head(2)

(1200, 15)


,Unnamed: 0,Title,Company,Location,Rating,Date,Salary,Description,Links,Descriptions,token_count,embedding,kmeans_label,llm_title,topic_title
0,0,Data Scientist,Driven Brands,"Benicia, CA",2.4,PostedPosted 26 days ago,None,You’ll be working alongside a team of eight an...,https://www.indeed.com/rc/clk?jk=74d176d595225...,We invite you to join us at Driven Brands!\nHe...,26,"[-0.0194996390491724, -0.0041993726044893265, ...",4,"""Join our dynamic team of analysts and data sc...",Data Science and Strategy Manager
1,1,Business Analyst,Sabot Consulting,Remote,NaN,PostedPosted 4 days ago,$80 - $120 an hour,Preferred candidates will have prior experienc...,https://www.indeed.com/rc/clk?jk=f662b2efb509b...,Sabot Consulting (Sabot) is a management consu...,25,"[-0.012073525227606297, -0.026480479165911674,...",2,"""Experienced Cloud Migration Specialist for SA...",IT Specialist


In [6]:
import sqlite3
from sqlalchemy import create_engine


In [7]:
def clean_names(df):
    df.columns = [x.replace(' ', '_').lower() for x in df.columns]
    return df

In [8]:
table_name = "jobs_table"
uri = "sqlite:///jobs_table.db"

In [9]:
# Create the sqlalchemy engine
engine = create_engine(uri, echo=False)
# Prep column names
data = clean_names(df)
# Convert the DataFrame to SQL
data.to_sql(table_name, con=engine, index=False, if_exists='replace')


1200

In [10]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.utilities import SQLDatabase
from langchain.chains import create_sql_query_chain
from langchain_experimental.sql import SQLDatabaseChain


In [11]:
db = SQLDatabase.from_uri("sqlite:///jobs_table.db")

In [12]:
chain = create_sql_query_chain(ChatOpenAI(model="gpt-3.5-turbo", temperature=0), db)

In [13]:
response = chain.invoke({"question":"How many companies are there"})
print(response)

SELECT COUNT(DISTINCT company) AS num_companies FROM jobs_table


In [14]:
db.run(response)

'[(883,)]'

In [15]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, verbose=False)
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, use_query_checker=True)

In [16]:
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

In [17]:
db_chain.run("How many distinct companies do not have remote based jobs?")



> Entering new SQLDatabaseChain chain...
How many distinct companies do not have remote based jobs?
SQLQuery:SELECT COUNT(DISTINCT company) 
FROM jobs_table 
WHERE location NOT LIKE '%Remote%'
SQLResult: [(371,)]
Answer:Final answer here: There are 371 distinct companies that do not have remote based jobs.
> Finished chain.


'Final answer here: There are 371 distinct companies that do not have remote based jobs.'

In [18]:
db_chain.run("What is the average rating for Remote, and what is the average rating for all other locations?")



> Entering new SQLDatabaseChain chain...
What is the average rating for Remote, and what is the average rating for all other locations?
SQLQuery:SELECT AVG(rating) AS avg_rating, 
       CASE WHEN location = 'Remote' THEN 'Remote' ELSE 'Other' END AS location_type
FROM jobs_table
GROUP BY location_type
SQLResult: [(3.7389324960753494, 'Other'), (3.6425925925925915, 'Remote')]
Answer:The average rating for Remote locations is 3.6426, and the average rating for all other locations is 3.7389.
> Finished chain.


'The average rating for Remote locations is 3.6426, and the average rating for all other locations is 3.7389.'

In [19]:
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, return_intermediate_steps=True)

In [20]:
result = db_chain("What percentage of jobs are listed as Remote based?")
result["intermediate_steps"]



> Entering new SQLDatabaseChain chain...
What percentage of jobs are listed as Remote based?
SQLQuery:SELECT COUNT(*) * 100.0 / (SELECT COUNT(*) FROM jobs_table) AS percentage
FROM jobs_table
WHERE location LIKE '%Remote%'
SQLResult: [(57.166666666666664,)]
Answer:The percentage of jobs listed as Remote based is approximately 57.17%.
> Finished chain.


[{'input': "What percentage of jobs are listed as Remote based?\nSQLQuery:SELECT COUNT(*) * 100.0 / (SELECT COUNT(*) FROM jobs_table) AS percentage\nFROM jobs_table\nWHERE location LIKE '%Remote%'\nSQLResult: [(57.166666666666664,)]\nAnswer:",
  'top_k': '5',
  'dialect': 'sqlite',
  'table_info': '\nCREATE TABLE jobs_table (\n\t"unnamed:_0" BIGINT, \n\ttitle TEXT, \n\tcompany TEXT, \n\tlocation TEXT, \n\trating FLOAT, \n\tdate TEXT, \n\tsalary TEXT, \n\tdescription TEXT, \n\tlinks TEXT, \n\tdescriptions TEXT, \n\ttoken_count BIGINT, \n\tembedding TEXT, \n\tkmeans_label INTEGER, \n\tllm_title TEXT, \n\ttopic_title TEXT\n)\n\n/*\n3 rows from jobs_table table:\nunnamed:_0\ttitle\tcompany\tlocation\trating\tdate\tsalary\tdescription\tlinks\tdescriptions\ttoken_count\tembedding\tkmeans_label\tllm_title\ttopic_title\n0\tData Scientist\tDriven Brands\tBenicia, CA\t2.4\tPostedPosted 26 days ago\tNone\tYou’ll be working alongside a team of eight analysts & data scientists - collaborating to de

In [21]:
result['result']

'The percentage of jobs listed as Remote based is approximately 57.17%.'

In [22]:
result = db_chain("Show the top 5 companies with the most jobs and their counts and average rating. Output in markdown table")



> Entering new SQLDatabaseChain chain...
Show the top 5 companies with the most jobs and their counts and average rating. Output in markdown table
SQLQuery:SELECT company, COUNT(*) AS job_count, AVG(rating) AS avg_rating
FROM jobs_table
GROUP BY company
ORDER BY job_count DESC
LIMIT 5;
SQLResult: [('Artech Consulting LLC', 10, None), ('CVS Health', 9, 3.3000000000000003), ('Sandhills Global, Inc.', 7, 3.1999999999999997), ('Ezoic', 7, 4.5), ('Capital One', 7, 3.8999999999999995)]
Answer:| Company                 | Job Count | Average Rating |
|-------------------------|-----------|----------------|
| Artech Consulting LLC   | 10        | None           |
| CVS Health              | 9         | 3.3            |
| Sandhills Global, Inc.  | 7         | 3.2            |
| Ezoic                   | 7         | 4.5            |
| Capital One             | 7         | 3.9            |
> Finished chain.


In [23]:
Markdown(f"{result['result']}")

| Company                 | Job Count | Average Rating |
|-------------------------|-----------|----------------|
| Artech Consulting LLC   | 10        | None           |
| CVS Health              | 9         | 3.3            |
| Sandhills Global, Inc.  | 7         | 3.2            |
| Ezoic                   | 7         | 4.5            |
| Capital One             | 7         | 3.9            |

### llama index

In [24]:
# NOTE: This is ONLY necessary in jupyter notebook.
# Details: Jupyter runs an event-loop behind the scenes.
#          This results in nested event-loops when we start an event-loop to make async queries.
#          This is normally not allowed, we use nest_asyncio to allow it for convenience.

import nest_asyncio
nest_asyncio.apply()

import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [25]:
from llama_index import (
    VectorStoreIndex,
    ServiceContext,
    StorageContext,
    SQLDatabase,
)

from llama_index.node_parser.simple import SimpleNodeParser
from llama_index import ServiceContext, LLMPredictor
from llama_index.storage import StorageContext
import chromadb
from llama_index.vector_stores import ChromaVectorStore
from langchain.text_splitter import TokenTextSplitter
from llama_index.llms import OpenAI

In [26]:
# define node parser and LLM
chunk_size = 512
llm = OpenAI(temperature=0, model="gpt-3.5-turbo", streaming=False)
service_context = ServiceContext.from_defaults(chunk_size=chunk_size, llm=llm)

text_splitter = TokenTextSplitter(chunk_size=chunk_size)
node_parser = SimpleNodeParser(text_splitter=text_splitter)

In [27]:
#create chroma client and collection
chroma_client = chromadb.Client()
chroma_collection = chroma_client.create_collection("jobs")

vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
vector_index = VectorStoreIndex([], storage_context=storage_context)

INFO:chromadb.telemetry.posthog:Anonymized telemetry enabled. See https://docs.trychroma.com/telemetry for more information.
Anonymized telemetry enabled. See https://docs.trychroma.com/telemetry for more information.


In [28]:
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    select,
    column,
)

In [29]:
#create SQL engine
engine = create_engine(uri, future=True)
metadata_obj = MetaData()
metadata_obj.drop_all(engine)

In [30]:
# Create table schema
indeed_jobs_table = Table(
    table_name,
    metadata_obj,
    Column("topic_title", String(50), primary_key=True),
    Column("location", String(20)),
    Column("company", String(20)),
    Column("title", String(20)),
    Column("rating", String(20))
)

In [31]:
metadata_obj.create_all(engine)

In [32]:
metadata_obj.tables.keys()

dict_keys(['jobs_table'])

In [33]:
sql_database = SQLDatabase(engine, include_tables=["jobs_table"])

In [34]:
from llama_index.indices.struct_store.sql_query import NLSQLTableQueryEngine
from llama_index.indices.struct_store.sql import SQLStructStoreIndex
from llama_index.schema import TextNode

In [35]:
sql_query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,
    tables=["jobs_table"],
)

In [36]:
sql_index = SQLStructStoreIndex.from_documents(
    [], 
    sql_database=sql_database, 
    table_name="jobs_table",
)

In [37]:
data.head(1)

,unnamed:_0,title,company,location,rating,date,salary,description,links,descriptions,token_count,embedding,kmeans_label,llm_title,topic_title
0,0,Data Scientist,Driven Brands,"Benicia, CA",2.4,PostedPosted 26 days ago,None,You’ll be working alongside a team of eight an...,https://www.indeed.com/rc/clk?jk=74d176d595225...,We invite you to join us at Driven Brands!\nHe...,26,"[-0.0194996390491724, -0.0041993726044893265, ...",4,"""Join our dynamic team of analysts and data sc...",Data Science and Strategy Manager


In [38]:
nodes = []

for index, row in data.iterrows():
    node = TextNode(text=row['descriptions'],
                            metadata={"topic_title": row['topic_title'],
                                      "location": row['location'],
                                      "company": row['company'],
                                      "title": row['title'],
                                      "rating": row['rating']})
    nodes.append(node)

In [39]:
len(nodes)

1200

In [40]:
vector_index.insert_nodes(nodes)

In [41]:
from llama_index.query_engine import SQLAutoVectorQueryEngine, RetrieverQueryEngine
from llama_index.tools.query_engine import QueryEngineTool
from llama_index.indices.vector_store import VectorIndexAutoRetriever

In [42]:
from llama_index.indices.vector_store.retrievers import VectorIndexAutoRetriever
from llama_index.vector_stores.types import MetadataInfo, VectorStoreInfo
from llama_index.query_engine.retriever_query_engine import RetrieverQueryEngine

vector_store_info = VectorStoreInfo(
    content_info="descriptions for job openings",
    metadata_info=[
        MetadataInfo(name="topic_title", type="str", description="The job category name"),
    ],
)
vector_auto_retriever = VectorIndexAutoRetriever(
    vector_index, vector_store_info=vector_store_info
)

retriever_query_engine = RetrieverQueryEngine.from_args(
    vector_auto_retriever, service_context=service_context
)

In [43]:
sql_tool = QueryEngineTool.from_defaults(
    query_engine=sql_query_engine,
    description=(
        "Useful for translating a natural language query into a SQL query over a table containing: "
        "jobs_table, containing structured data for job openings."
    ),
)
vector_tool = QueryEngineTool.from_defaults(
    query_engine=retriever_query_engine,
    description=f"Useful for answering semantic questions about different job descriptions",
)

In [44]:
query_engine = SQLAutoVectorQueryEngine(
    sql_tool, vector_tool, service_context=service_context
)

In [45]:
response = query_engine.query(
    "Summarize the job descriptions for the company with the most listings"
)

Querying SQL database: Useful for translating a natural language query into a SQL query over a table containing job descriptions.
INFO:llama_index.query_engine.sql_join_query_engine:> Querying SQL database: Useful for translating a natural language query into a SQL query over a table containing job descriptions.
> Querying SQL database: Useful for translating a natural language query into a SQL query over a table containing job descriptions.
INFO:llama_index.indices.struct_store.sql_query:> Table desc str: Table 'jobs_table' has columns: unnamed:_0 (BIGINT), title (TEXT), company (TEXT), location (TEXT), rating (FLOAT), date (TEXT), salary (TEXT), description (TEXT), links (TEXT), descriptions (TEXT), token_count (BIGINT), embedding (TEXT), kmeans_label (INTEGER), llm_title (TEXT), topic_title (TEXT) and foreign keys: .
> Table desc str: Table 'jobs_table' has columns: unnamed:_0 (BIGINT), title (TEXT), company (TEXT), location (TEXT), rating (FLOAT), date (TEXT), salary (TEXT), descri

In [46]:
Markdown(f"{response}")


Artech Consulting LLC has the most job listings with 10 total. Job descriptions for this company include roles in software engineering, data analysis, project management, and customer service.

In [49]:
response = query_engine.query(
    "Which job category name is the most common?"
)

Querying SQL database: Useful for translating a natural language query into a SQL query over a table containing: jobs_table, containing structured data for job openings.
INFO:llama_index.query_engine.sql_join_query_engine:> Querying SQL database: Useful for translating a natural language query into a SQL query over a table containing: jobs_table, containing structured data for job openings.
> Querying SQL database: Useful for translating a natural language query into a SQL query over a table containing: jobs_table, containing structured data for job openings.
INFO:llama_index.indices.struct_store.sql_query:> Table desc str: Table 'jobs_table' has columns: unnamed:_0 (BIGINT), title (TEXT), company (TEXT), location (TEXT), rating (FLOAT), date (TEXT), salary (TEXT), description (TEXT), links (TEXT), descriptions (TEXT), token_count (BIGINT), embedding (TEXT), kmeans_label (INTEGER), llm_title (TEXT), topic_title (TEXT) and foreign keys: .
> Table desc str: Table 'jobs_table' has columns

In [50]:
Markdown(f"{response}")

 The most common job category name is 4, with 281 occurrences.